# Lithium Iron Phosphate Battery Degradation Models for Utility-Scale Applications

This notebook demonstrates the implementation and analysis of LFP battery degradation models for utility-scale energy storage applications. The models account for both calendar aging (time-dependent degradation during rest) and cycle aging (usage-dependent degradation) with temperature dependencies.

## Key Features:
- Calendar and cycle aging models for LFP batteries
- Temperature-dependent degradation (Arrhenius scaling)
- Utility-scale simulation (daily 100% DoD cycling)
- State of Health (SoH) tracking and end-of-life prediction
- Model validation and comparative analysis
- Comprehensive visualization of results

## Scientific Background:
LFP batteries are increasingly used in utility-scale energy storage due to their excellent cycle life and stability. The primary degradation mechanism is SEI growth on the graphite anode, consuming cyclable lithium. This notebook implements models based on recent literature to predict capacity fade under realistic grid operation conditions.

## 1. Import Required Libraries and Dependencies

In [ ]:
# Core scientific computing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration
np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"SciPy version: {scipy.__version__}" if 'scipy' in locals() else "SciPy imported")

## 2. Define Battery Parameters and Constants

In [ ]:
# Physical Constants
R_GAS = 8.314  # Gas constant [J/(mol·K)]
K_TO_C = 273.15  # Kelvin to Celsius conversion

# LFP Battery Parameters
class LFPBatteryParams:
    """Parameters for LFP/Graphite battery cells"""
    
    def __init__(self):
        # Basic battery properties
        self.initial_capacity = 100.0  # Initial capacity [Ah]
        self.nominal_voltage = 3.3  # Nominal voltage [V]
        self.eol_threshold = 80.0  # End-of-life threshold [%]
        
        # Calendar aging parameters
        self.A_cal = 0.075  # Pre-exponential factor for calendar aging [%/sqrt(year)]
        self.Ea_cal = 0.65  # Activation energy for calendar aging [eV]
        self.alpha_soc = 0.3  # SoC stress coefficient [-]
        
        # Cycle aging parameters
        self.B_cyc = 5e-5  # Cycle aging coefficient [%/cycle^beta]
        self.beta = 0.75  # Power law exponent [-]
        self.Ea_cyc = 0.35  # Activation energy for cycle aging [eV]
        self.gamma_dod = 1.1  # DoD stress exponent [-]
        self.delta_crate = 0.15  # C-rate stress coefficient [-]
        
        # Operating conditions (defaults)
        self.temperature_ref = 25.0  # Reference temperature [°C]
        self.soc_avg = 0.5  # Average state of charge [-]
        self.dod = 1.0  # Depth of discharge (100% for utility-scale) [-]
        self.c_rate = 1.0  # Charge/discharge rate [C]
        
        # Utility-scale operation
        self.cycles_per_day = 1  # One full cycle per day
        self.days_per_year = 365
        
    def display(self):
        """Display battery parameters"""
        print("="*60)
        print("LFP Battery Parameters".center(60))
        print("="*60)
        print(f"\nBasic Properties:")
        print(f"  Initial Capacity: {self.initial_capacity} Ah")
        print(f"  Nominal Voltage: {self.nominal_voltage} V")
        print(f"  End-of-Life: {self.eol_threshold}% capacity")
        print(f"\nCalendar Aging:")
        print(f"  Pre-exponential factor (A_cal): {self.A_cal} %/sqrt(year)")
        print(f"  Activation energy (Ea_cal): {self.Ea_cal} eV")
        print(f"  SoC stress coefficient (α): {self.alpha_soc}")
        print(f"\nCycle Aging:")
        print(f"  Cycle coefficient (B_cyc): {self.B_cyc:.2e} %/cycle^β")
        print(f"  Power law exponent (β): {self.beta}")
        print(f"  Activation energy (Ea_cyc): {self.Ea_cyc} eV")
        print(f"\nUtility-Scale Operation:")
        print(f"  Cycles per day: {self.cycles_per_day}")
        print(f"  Depth of Discharge: {self.dod*100}%")
        print(f"  C-rate: {self.c_rate}C")
        print("="*60)

# Initialize parameters
params = LFPBatteryParams()
params.display()

## 3. Implement Calendar Aging Models

Calendar aging represents time-dependent degradation that occurs even during storage. The primary mechanism is SEI growth on the graphite anode.

In [ ]:
def arrhenius_factor(temperature_c, temperature_ref_c, activation_energy_ev):
    """
    Calculate Arrhenius temperature scaling factor.
    
    Parameters:
    -----------
    temperature_c : float
        Operating temperature [°C]
    temperature_ref_c : float
        Reference temperature [°C]
    activation_energy_ev : float
        Activation energy [eV]
    
    Returns:
    --------
    factor : float
        Temperature scaling factor (dimensionless)
    """
    T = temperature_c + K_TO_C  # Convert to Kelvin
    T_ref = temperature_ref_c + K_TO_C
    
    # Arrhenius equation: exp(-Ea/(R*T))
    # We normalize to reference temperature
    factor = np.exp(activation_energy_ev * 11605 * (1/T_ref - 1/T))
    # Note: 11605 = e/(k_B) converts eV to K
    
    return factor

def soc_stress_factor(soc, alpha):
    """
    Calculate SoC-dependent stress factor.
    
    Parameters:
    -----------
    soc : float or array
        State of charge [0-1]
    alpha : float
        SoC stress coefficient
    
    Returns:
    --------
    factor : float or array
        SoC stress factor (dimensionless)
    """
    return 1 + alpha * (soc - 0.5)**2

def calendar_aging_loss(time_years, temperature_c, soc, params):
    """
    Calculate calendar aging capacity loss.
    
    Parameters:
    -----------
    time_years : float or array
        Time elapsed [years]
    temperature_c : float
        Operating temperature [°C]
    soc : float
        Average state of charge [0-1]
    params : LFPBatteryParams
        Battery parameters
    
    Returns:
    --------
    capacity_loss : float or array
        Calendar capacity loss [%]
    """
    # Temperature dependence (Arrhenius)
    temp_factor = arrhenius_factor(temperature_c, params.temperature_ref, params.Ea_cal)
    
    # SoC dependence
    soc_factor = soc_stress_factor(soc, params.alpha_soc)
    
    # Square root of time dependence (SEI growth kinetics)
    capacity_loss = params.A_cal * np.sqrt(time_years) * temp_factor * soc_factor
    
    return capacity_loss

# Test calendar aging at different temperatures
print("Calendar Aging Test:")
print("-" * 60)
test_years = np.array([1, 5, 10, 15, 20])
test_temps = [25, 40, 55]

for temp in test_temps:
    losses = calendar_aging_loss(test_years, temp, 0.5, params)
    print(f"\nTemperature: {temp}°C")
    for year, loss in zip(test_years, losses):
        print(f"  Year {year:2d}: {loss:5.2f}% capacity loss")
        
# Calculate acceleration factors
accel_40 = arrhenius_factor(40, 25, params.Ea_cal)
accel_55 = arrhenius_factor(55, 25, params.Ea_cal)
print(f"\n{'='*60}")
print(f"Temperature Acceleration Factors (vs 25°C):")
print(f"  40°C: {accel_40:.2f}× faster aging")
print(f"  55°C: {accel_55:.2f}× faster aging")
print(f"{'='*60}")

## 4. Implement Cycle Aging Models

Cycle aging represents usage-dependent degradation from charge-discharge cycles.

In [ ]:
def dod_stress_factor(dod, gamma):
    """
    Calculate DoD-dependent stress factor.
    
    Parameters:
    -----------
    dod : float or array
        Depth of discharge [0-1]
    gamma : float
        DoD stress exponent
    
    Returns:
    --------
    factor : float or array
        DoD stress factor (dimensionless)
    """
    return dod**gamma

def crate_stress_factor(c_rate, delta):
    """
    Calculate C-rate stress factor.
    
    Parameters:
    -----------
    c_rate : float or array
        Charge/discharge rate [C]
    delta : float
        C-rate stress coefficient
    
    Returns:
    --------
    factor : float or array
        C-rate stress factor (dimensionless)
    """
    return 1 + delta * np.maximum(0, c_rate - 1)

def cycle_aging_loss(num_cycles, temperature_c, dod, c_rate, params):
    """
    Calculate cycle aging capacity loss.
    
    Parameters:
    -----------
    num_cycles : float or array
        Number of cycles
    temperature_c : float
        Operating temperature [°C]
    dod : float
        Depth of discharge [0-1]
    c_rate : float
        Charge/discharge rate [C]
    params : LFPBatteryParams
        Battery parameters
    
    Returns:
    --------
    capacity_loss : float or array
        Cycle capacity loss [%]
    """
    # Temperature dependence
    temp_factor = arrhenius_factor(temperature_c, params.temperature_ref, params.Ea_cyc)
    
    # DoD dependence
    dod_factor = dod_stress_factor(dod, params.gamma_dod)
    
    # C-rate dependence
    crate_factor = crate_stress_factor(c_rate, params.delta_crate)
    
    # Power law dependence on cycle number
    capacity_loss = params.B_cyc * (num_cycles**params.beta) * temp_factor * dod_factor * crate_factor
    
    return capacity_loss

# Test cycle aging
print("Cycle Aging Test:")
print("-" * 60)
test_cycles = np.array([365, 365*5, 365*10, 365*15, 365*20])  # Daily cycling for years

for temp in [25, 40, 55]:
    losses = cycle_aging_loss(test_cycles, temp, 1.0, 1.0, params)
    print(f"\nTemperature: {temp}°C (100% DoD, 1C rate)")
    for cycles, loss, years in zip(test_cycles, losses, [1, 5, 10, 15, 20]):
        print(f"  Year {years:2d} ({cycles:5d} cycles): {loss:5.2f}% capacity loss")

# Test DoD impact
print(f"\n{'='*60}")
print("DoD Impact (25°C, 1C, 10 years = 3650 cycles):")
test_dods = [0.5, 0.8, 1.0]
for dod in test_dods:
    loss = cycle_aging_loss(3650, 25, dod, 1.0, params)
    print(f"  {dod*100:3.0f}% DoD: {loss:5.2f}% capacity loss")
print(f"{'='*60}")

## 5. Combined Degradation Model

Integrate calendar and cycle aging into a unified framework.

In [ ]:
class LFPDegradationModel:
    """
    Combined degradation model for LFP batteries.
    Accounts for both calendar and cycle aging.
    """
    
    def __init__(self, params):
        self.params = params
        
    def total_capacity_loss(self, time_years, temperature_c, soc=0.5, 
                           dod=1.0, c_rate=1.0, cycles_per_day=1):
        """
        Calculate total capacity loss from combined aging mechanisms.
        
        Parameters:
        -----------
        time_years : float or array
            Time elapsed [years]
        temperature_c : float
            Operating temperature [°C]
        soc : float
            Average state of charge [0-1]
        dod : float
            Depth of discharge [0-1]
        c_rate : float
            Charge/discharge rate [C]
        cycles_per_day : int
            Number of cycles per day
        
        Returns:
        --------
        total_loss : float or array
            Total capacity loss [%]
        cal_loss : float or array
            Calendar aging component [%]
        cyc_loss : float or array
            Cycle aging component [%]
        """
        # Calendar aging
        cal_loss = calendar_aging_loss(time_years, temperature_c, soc, self.params)
        
        # Cycle aging (convert years to cycles)
        num_cycles = time_years * self.params.days_per_year * cycles_per_day
        cyc_loss = cycle_aging_loss(num_cycles, temperature_c, dod, c_rate, self.params)
        
        # Total loss (superposition)
        total_loss = cal_loss + cyc_loss
        
        return total_loss, cal_loss, cyc_loss
    
    def state_of_health(self, time_years, temperature_c, soc=0.5, 
                       dod=1.0, c_rate=1.0, cycles_per_day=1):
        """
        Calculate State of Health (SoH).
        
        Returns:
        --------
        soh : float or array
            State of health [%], where 100% is pristine
        """
        total_loss, _, _ = self.total_capacity_loss(
            time_years, temperature_c, soc, dod, c_rate, cycles_per_day
        )
        soh = 100 - total_loss
        return soh
    
    def end_of_life_years(self, temperature_c, soc=0.5, dod=1.0, 
                         c_rate=1.0, cycles_per_day=1):
        """
        Estimate time to reach end-of-life (80% SoH).
        
        Returns:
        --------
        eol_years : float
            Years until end-of-life
        """
        # Search for time when SoH reaches 80%
        time_test = np.linspace(0, 30, 1000)  # Test up to 30 years
        soh = self.state_of_health(time_test, temperature_c, soc, dod, c_rate, cycles_per_day)
        
        # Find first time SoH drops below 80%
        idx = np.where(soh <= self.params.eol_threshold)[0]
        if len(idx) > 0:
            eol_years = time_test[idx[0]]
        else:
            eol_years = np.nan  # Exceeds 30 years
        
        return eol_years

# Initialize model
model = LFPDegradationModel(params)

# Test combined model
print("Combined Degradation Model Test:")
print("="*60)
test_years = np.array([1, 5, 10, 15, 20])

for temp in [25, 40, 55]:
    print(f"\nTemperature: {temp}°C (Daily 100% DoD cycling)")
    print(f"{'Year':<6} {'Total Loss':>12} {'Calendar':>10} {'Cycle':>10} {'SoH':>8}")
    print("-"*50)
    
    total, cal, cyc = model.total_capacity_loss(test_years, temp, 0.5, 1.0, 1.0, 1)
    soh = model.state_of_health(test_years, temp, 0.5, 1.0, 1.0, 1)
    
    for y, t, c1, c2, s in zip(test_years, total, cal, cyc, soh):
        print(f"{y:<6} {t:>11.2f}% {c1:>9.2f}% {c2:>9.2f}% {s:>7.1f}%")
    
    eol = model.end_of_life_years(temp, 0.5, 1.0, 1.0, 1)
    print(f"\nEnd-of-Life: {eol:.1f} years" if not np.isnan(eol) else "\nEnd-of-Life: >30 years")

print("="*60)

## 6. Visualization and Results Analysis

Create comprehensive visualizations of degradation behavior.

In [ ]:
# Generate time series data
time_years = np.linspace(0, 20, 200)
temperatures = [20, 25, 30, 35, 40, 45, 50, 55]

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('LFP Battery Degradation Analysis - Utility-Scale Operation', 
             fontsize=16, fontweight='bold')

# Plot 1: SoH vs Time for different temperatures
ax1 = axes[0, 0]
colors = plt.cm.coolwarm(np.linspace(0, 1, len(temperatures)))

for temp, color in zip(temperatures, colors):
    soh = model.state_of_health(time_years, temp, 0.5, 1.0, 1.0, 1)
    ax1.plot(time_years, soh, label=f'{temp}°C', linewidth=2, color=color)

ax1.axhline(y=80, color='red', linestyle='--', linewidth=2, alpha=0.7, label='EOL (80%)')
ax1.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax1.set_ylabel('State of Health (%)', fontsize=12, fontweight='bold')
ax1.set_title('Capacity Retention vs Time', fontsize=13, fontweight='bold')
ax1.legend(loc='best', ncol=2, fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 20)
ax1.set_ylim(70, 105)

# Plot 2: Calendar vs Cycle contributions
ax2 = axes[0, 1]
temp_analysis = 25  # Room temperature

total_loss, cal_loss, cyc_loss = model.total_capacity_loss(
    time_years, temp_analysis, 0.5, 1.0, 1.0, 1
)

ax2.fill_between(time_years, 0, cal_loss, alpha=0.6, label='Calendar Aging', color='steelblue')
ax2.fill_between(time_years, cal_loss, cal_loss + cyc_loss, alpha=0.6, 
                 label='Cycle Aging', color='coral')
ax2.plot(time_years, total_loss, 'k-', linewidth=2.5, label='Total Loss')

ax2.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Capacity Loss (%)', fontsize=12, fontweight='bold')
ax2.set_title(f'Aging Components Breakdown ({temp_analysis}°C)', fontsize=13, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 20)

# Plot 3: End-of-Life vs Temperature
ax3 = axes[1, 0]
temp_range = np.linspace(15, 60, 50)
eol_years = []

for temp in temp_range:
    eol = model.end_of_life_years(temp, 0.5, 1.0, 1.0, 1)
    eol_years.append(eol if not np.isnan(eol) else 30)

ax3.plot(temp_range, eol_years, linewidth=3, color='darkgreen')
ax3.fill_between(temp_range, eol_years, alpha=0.3, color='lightgreen')
ax3.axhline(y=20, color='orange', linestyle='--', linewidth=2, alpha=0.7, 
            label='20-year target')
ax3.axvline(x=25, color='blue', linestyle=':', linewidth=2, alpha=0.5, 
            label='Room temp (25°C)')

ax3.set_xlabel('Operating Temperature (°C)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Battery Life (years to 80% SoH)', fontsize=12, fontweight='bold')
ax3.set_title('Temperature Impact on Battery Lifetime', fontsize=13, fontweight='bold')
ax3.legend(loc='upper right', fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(15, 60)

# Plot 4: Temperature acceleration factors
ax4 = axes[1, 1]
accel_factors = [arrhenius_factor(temp, 25, params.Ea_cal) for temp in temp_range]

ax4.semilogy(temp_range, accel_factors, linewidth=3, color='crimson')
ax4.fill_between(temp_range, 1, accel_factors, alpha=0.3, color='lightcoral')
ax4.axhline(y=1, color='blue', linestyle='--', linewidth=2, alpha=0.7, 
            label='Baseline (25°C)')
ax4.axhline(y=1.3, color='orange', linestyle=':', linewidth=2, alpha=0.7, 
            label='1.3× (40°C)')
ax4.axhline(y=5, color='red', linestyle=':', linewidth=2, alpha=0.7, 
            label='5× (55°C)')

ax4.set_xlabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Aging Rate (relative to 25°C)', fontsize=12, fontweight='bold')
ax4.set_title('Temperature Acceleration Factor (Arrhenius)', fontsize=13, fontweight='bold')
ax4.legend(loc='upper left', fontsize=10)
ax4.grid(True, alpha=0.3, which='both')
ax4.set_xlim(15, 60)
ax4.set_ylim(0.5, 20)

plt.tight_layout()
plt.show()

print("\nVisualization complete!")

## 7. Utility-Scale Simulation and Analysis

Create a detailed simulation for utility-scale battery operation with realistic operating profiles.

In [ ]:
# Create utility-scale scenarios
scenarios = {
    'Conservative (20°C)': {'temp': 20, 'color': 'blue'},
    'Standard (25°C)': {'temp': 25, 'color': 'green'},
    'Warm Climate (30°C)': {'temp': 30, 'color': 'orange'},
    'Hot Climate (35°C)': {'temp': 35, 'color': 'red'},
}

# Simulation parameters
sim_years = 20
sim_time = np.linspace(0, sim_years, sim_years * 12)  # Monthly resolution

# Create comparison dataframe
results_df = pd.DataFrame()

for scenario_name, scenario_params in scenarios.items():
    temp = scenario_params['temp']
    
    # Calculate degradation
    total_loss, cal_loss, cyc_loss = model.total_capacity_loss(
        sim_time, temp, 0.5, 1.0, 1.0, 1
    )
    soh = 100 - total_loss
    
    # Store results
    scenario_df = pd.DataFrame({
        'Time (years)': sim_time,
        'Temperature (°C)': temp,
        'SoH (%)': soh,
        'Total Loss (%)': total_loss,
        'Calendar Loss (%)': cal_loss,
        'Cycle Loss (%)': cyc_loss,
        'Scenario': scenario_name
    })
    
    results_df = pd.concat([results_df, scenario_df], ignore_index=True)

# Display summary statistics
print("="*80)
print("UTILITY-SCALE LFP BATTERY DEGRADATION ANALYSIS")
print("="*80)
print("\nOperating Conditions:")
print("  - Daily cycling: 1 full cycle per day (100% DoD)")
print("  - C-rate: 1C charge/discharge")
print("  - Analysis period: 20 years")
print("\n" + "="*80)
print(f"{'Scenario':<25} {'Temp':<8} {'SoH@10yr':<12} {'SoH@20yr':<12} {'EOL (years)':<15}")
print("-"*80)

for scenario_name, scenario_params in scenarios.items():
    temp = scenario_params['temp']
    
    # Get SoH at key milestones
    soh_10yr = model.state_of_health(10, temp, 0.5, 1.0, 1.0, 1)
    soh_20yr = model.state_of_health(20, temp, 0.5, 1.0, 1.0, 1)
    eol = model.end_of_life_years(temp, 0.5, 1.0, 1.0, 1)
    
    eol_str = f"{eol:.1f}" if not np.isnan(eol) else ">30"
    print(f"{scenario_name:<25} {temp:<8} {soh_10yr:>10.1f}% {soh_20yr:>10.1f}% {eol_str:>13}")

print("="*80)

# Detailed visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Utility-Scale Battery System Performance Comparison', 
             fontsize=16, fontweight='bold')

# Plot 1: SoH comparison
ax1 = axes[0, 0]
for scenario_name, scenario_params in scenarios.items():
    scenario_data = results_df[results_df['Scenario'] == scenario_name]
    ax1.plot(scenario_data['Time (years)'], scenario_data['SoH (%)'], 
            label=scenario_name, linewidth=2.5, color=scenario_params['color'])

ax1.axhline(y=80, color='black', linestyle='--', linewidth=2, alpha=0.7, label='EOL Threshold')
ax1.fill_between([0, 20], 80, 70, alpha=0.2, color='red', label='Below EOL')
ax1.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax1.set_ylabel('State of Health (%)', fontsize=12, fontweight='bold')
ax1.set_title('Battery Capacity Retention', fontsize=13, fontweight='bold')
ax1.legend(loc='lower left', fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 20)
ax1.set_ylim(70, 102)

# Plot 2: Degradation rate
ax2 = axes[0, 1]
for scenario_name, scenario_params in scenarios.items():
    scenario_data = results_df[results_df['Scenario'] == scenario_name]
    ax2.plot(scenario_data['Time (years)'], scenario_data['Total Loss (%)'], 
            label=scenario_name, linewidth=2.5, color=scenario_params['color'])

ax2.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cumulative Capacity Loss (%)', fontsize=12, fontweight='bold')
ax2.set_title('Degradation Progression', fontsize=13, fontweight='bold')
ax2.legend(loc='upper left', fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 20)

# Plot 3: Aging breakdown for 25°C case
ax3 = axes[1, 0]
standard_data = results_df[results_df['Scenario'] == 'Standard (25°C)']
ax3.fill_between(standard_data['Time (years)'], 0, standard_data['Calendar Loss (%)'], 
                 alpha=0.6, label='Calendar Aging', color='steelblue')
ax3.fill_between(standard_data['Time (years)'], 
                 standard_data['Calendar Loss (%)'], 
                 standard_data['Total Loss (%)'], 
                 alpha=0.6, label='Cycle Aging', color='coral')
ax3.plot(standard_data['Time (years)'], standard_data['Total Loss (%)'], 
         'k-', linewidth=2, label='Total')

ax3.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Capacity Loss (%)', fontsize=12, fontweight='bold')
ax3.set_title('Degradation Mechanism Breakdown (25°C)', fontsize=13, fontweight='bold')
ax3.legend(loc='upper left', fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 20)

# Plot 4: Annual degradation rate
ax4 = axes[1, 1]
for scenario_name, scenario_params in scenarios.items():
    scenario_data = results_df[results_df['Scenario'] == scenario_name].copy()
    # Calculate year-over-year degradation rate
    scenario_data['Annual Rate (%)'] = scenario_data['Total Loss (%)'].diff() / \
                                       scenario_data['Time (years)'].diff()
    scenario_data = scenario_data[scenario_data['Time (years)'] > 0.5]  # Skip initial
    
    ax4.plot(scenario_data['Time (years)'], scenario_data['Annual Rate (%)'], 
            label=scenario_name, linewidth=2, color=scenario_params['color'], alpha=0.7)

ax4.set_xlabel('Time (years)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Annual Degradation Rate (%/year)', fontsize=12, fontweight='bold')
ax4.set_title('Time-Varying Degradation Rate', fontsize=13, fontweight='bold')
ax4.legend(loc='upper right', fontsize=9)
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 20)

plt.tight_layout()
plt.show()

print("\nSimulation analysis complete!")

## 8. Summary and Key Findings

Generate summary statistics and export results.

In [ ]:
# Generate comprehensive summary report
print("\n" + "="*80)
print("LFP BATTERY DEGRADATION MODEL - EXECUTIVE SUMMARY")
print("="*80)

print("\n1. MODEL CHARACTERISTICS:")
print("-" * 80)
print(f"   Chemistry: Lithium Iron Phosphate (LFP) / Graphite")
print(f"   Application: Utility-scale energy storage (daily cycling)")
print(f"   Operating Profile: 1 cycle/day at 100% DoD")
print(f"   Initial Capacity: {params.initial_capacity} Ah")
print(f"   End-of-Life Criterion: {params.eol_threshold}% capacity retention")

print("\n2. KEY DEGRADATION MECHANISMS:")
print("-" * 80)
print(f"   Primary: SEI growth on graphite anode (calendar + cycle aging)")
print(f"   Calendar Aging Activation Energy: {params.Ea_cal} eV")
print(f"   Cycle Aging Activation Energy: {params.Ea_cyc} eV")
print(f"   Time Dependence: Square-root (SEI growth kinetics)")
print(f"   Cycle Dependence: Power law with exponent β = {params.beta}")

print("\n3. TEMPERATURE SENSITIVITY:")
print("-" * 80)
temp_comparisons = [
    (25, 1.0),
    (30, arrhenius_factor(30, 25, params.Ea_cal)),
    (35, arrhenius_factor(35, 25, params.Ea_cal)),
    (40, arrhenius_factor(40, 25, params.Ea_cal)),
    (45, arrhenius_factor(45, 25, params.Ea_cal)),
    (55, arrhenius_factor(55, 25, params.Ea_cal)),
]

for temp, factor in temp_comparisons:
    eol = model.end_of_life_years(temp, 0.5, 1.0, 1.0, 1)
    eol_str = f"{eol:.1f} years" if not np.isnan(eol) else ">30 years"
    print(f"   {temp}°C: {factor:.2f}× aging rate | Expected life: {eol_str}")

print("\n4. DEGRADATION CONTRIBUTIONS (25°C, 20 years):")
print("-" * 80)
total_20yr, cal_20yr, cyc_20yr = model.total_capacity_loss(20, 25, 0.5, 1.0, 1.0, 1)
cal_pct = (cal_20yr / total_20yr) * 100
cyc_pct = (cyc_20yr / total_20yr) * 100

print(f"   Total Capacity Loss: {total_20yr:.2f}%")
print(f"   Calendar Component: {cal_20yr:.2f}% ({cal_pct:.1f}% of total)")
print(f"   Cycle Component: {cyc_20yr:.2f}% ({cyc_pct:.1f}% of total)")
print(f"   Final SoH: {100 - total_20yr:.1f}%")

print("\n5. UTILITY-SCALE PERFORMANCE PROJECTIONS:")
print("-" * 80)
scenarios_summary = [
    ('Conservative (20°C)', 20),
    ('Standard (25°C)', 25),
    ('Warm Climate (30°C)', 30),
    ('Hot Climate (35°C)', 35),
]

print(f"   {'Scenario':<25} {'10-yr SoH':<12} {'20-yr SoH':<12} {'Warranty Met?'}")
print("   " + "-"*65)

for name, temp in scenarios_summary:
    soh_10 = model.state_of_health(10, temp, 0.5, 1.0, 1.0, 1)
    soh_20 = model.state_of_health(20, temp, 0.5, 1.0, 1.0, 1)
    warranty_10 = "✓ Yes" if soh_10 >= 80 else "✗ No"
    warranty_20 = "✓ Yes" if soh_20 >= 80 else "✗ No"
    print(f"   {name:<25} {soh_10:>8.1f}%    {soh_20:>8.1f}%    10yr:{warranty_10}  20yr:{warranty_20}")

print("\n6. RECOMMENDATIONS:")
print("-" * 80)
print("   • Temperature control is critical - each 10°C increase roughly doubles aging rate")
print("   • LFP batteries can meet 20-year utility warranties at temperatures ≤30°C")
print("   • Calendar aging dominates at low cycle rates (1 cycle/day)")
print("   • Active thermal management recommended for hot climates (>35°C ambient)")
print("   • Consider SOC management strategies to reduce calendar aging stress")

print("\n7. MODEL LIMITATIONS:")
print("-" * 80)
print("   • Assumes constant operating conditions (temperature, DoD, C-rate)")
print("   • Does not account for cell-to-cell variations")
print("   • Limited validation data for extreme conditions (>55°C, <0°C)")
print("   • Does not model lithium plating at low temperatures")
print("   • Superposition may underestimate interactions at high stress levels")

print("\n" + "="*80)
print("Analysis complete. Results saved to results_df DataFrame.")
print("="*80 + "\n")

# Export summary to CSV
summary_file = 'lfp_degradation_summary.csv'
results_df.to_csv(summary_file, index=False)
print(f"✓ Detailed results exported to: {summary_file}")

# Create a summary statistics table
summary_stats = results_df.groupby('Scenario').agg({
    'SoH (%)': ['min', 'mean', 'std'],
    'Total Loss (%)': ['max', 'mean'],
    'Temperature (°C)': 'first'
}).round(2)

print(f"\n✓ Summary statistics generated")
print(summary_stats)